In [54]:
import re
import networkx as nx
from itertools import permutations

filename = './input.txt'
#filename = './test.txt'

with open(filename, 'r') as f:
    sequence = f.read().split('\n')

kpad = ['789',
        '456',
        '123',
        ' 0A']

dpad = [' ^A',
        '<v>']

K = nx.DiGraph()
D = nx.DiGraph()
K.add_edge
for r in range(3):
    K.add_edge(kpad[r][0], kpad[r][1], drx='>')
    K.add_edge(kpad[r][1], kpad[r][2], drx='>')
    K.add_edge(kpad[r][1], kpad[r][0], drx='<')
    K.add_edge(kpad[r][2], kpad[r][1], drx='<')

for c in range(3):
    K.add_edge(kpad[0][c], kpad[1][c], drx='v')
    K.add_edge(kpad[1][c], kpad[2][c], drx='v')
    K.add_edge(kpad[1][c], kpad[0][c], drx='^')
    K.add_edge(kpad[2][c], kpad[1][c], drx='^')

K.add_edge(kpad[2][1], kpad[3][1], drx='v')
K.add_edge(kpad[2][2], kpad[3][2], drx='v')
K.add_edge(kpad[3][1], kpad[3][2], drx='>')
K.add_edge(kpad[3][1], kpad[2][1], drx='^')
K.add_edge(kpad[3][2], kpad[2][2], drx='^')
K.add_edge(kpad[3][2], kpad[3][1], drx='<')

D.add_edge(dpad[0][1], dpad[0][2], drx='>')
D.add_edge(dpad[1][0], dpad[1][1], drx='>')
D.add_edge(dpad[1][1], dpad[1][2], drx='>')
D.add_edge(dpad[0][1], dpad[1][1], drx='v')
D.add_edge(dpad[0][2], dpad[1][2], drx='v')
D.add_edge(dpad[0][2], dpad[0][1], drx='<')
D.add_edge(dpad[1][1], dpad[1][0], drx='<')
D.add_edge(dpad[1][2], dpad[1][1], drx='<')
D.add_edge(dpad[1][1], dpad[0][1], drx='^')
D.add_edge(dpad[1][2], dpad[0][2], drx='^')

# generate shortest paths/moves keypad
__kcombos = permutations('A0123456789', 2)
keypaths = {}
for u, v in __kcombos:
    keypaths[(u, v)] = [*nx.all_shortest_paths(K, u, v)]

kpathMoves = {}
for (u, v), paths in keypaths.items():
    kpathMoves[f"{u}{v}"] = []
    for p in paths:
        pstr = ''
        i = 0
        while i+1 < len(p):
            pstr += K[p[i]][p[i+1]]['drx']
            i += 1
        kpathMoves[f"{u}{v}"].append(pstr)

# generate shortest paths/moves dpad
__dcombos = permutations('<v>^A', 2)
drxpaths = {}
for u, v in __dcombos:
    drxpaths[(u, v)] = [*nx.all_shortest_paths(D, u, v)]

dpathMoves = {}
for (u, v), paths in drxpaths.items():
    dpathMoves[f"{u}{v}"] = []
    for p in paths:
        pstr = ''
        i = 0
        while i+1 < len(p):
            pstr += D[p[i]][p[i+1]]['drx']
            i += 1
        dpathMoves[f"{u}{v}"].append(pstr)

def minDirChange(seq):
    count = 0
    prev = seq[0]
    for s in seq[1:]:
        if prev != s:
            count += 1
        prev = s
    return count

def prune(paths):
    if len(paths) == 1:
        return paths
    else:
        changes = []
        shortestpaths = []
        for p in paths:
            changes.append(minDirChange(p))
        minChange = min(changes)
        for idx, n in enumerate(changes):
            if n == minChange:
                shortestpaths.append(paths[idx])
        return shortestpaths

for k, moves in kpathMoves.items():
    kpathMoves[k] = prune(moves)

for k, moves in dpathMoves.items():
    dpathMoves[k] = prune(moves)

###

def buildKeypadSeq(code, index, prevKey, currPath, result):
    if index == len(code):
        result.append(currPath)
        return
    for path in kpathMoves.get(f"{prevKey}{code[index]}", ''):
        buildKeypadSeq(code, index+1, code[index], currPath + path + 'A', result)

def buildSeq(seq, index, prevKey, currPath, result):
    if index == len(seq):
        result.append(currPath)
        return
    if prevKey == seq[index]:
        buildSeq(seq, index+1, seq[index], currPath + 'A', result)
    else:
        for path in dpathMoves.get(f"{prevKey}{seq[index]}", 'A'):
            buildSeq(seq, index+1, seq[index], currPath + path + 'A', result)

memo = {}
def shortestSeq(keys, depth):
    if (keys, depth) in memo:
        return memo[(keys, depth)]
    if depth == 0:
        return len(keys)
    total = 0
    idx = []
    for i, c in enumerate(keys):
        if c == 'A':
            idx.append(i)
    prev = 0
    for i in idx:
        result = []
        buildSeq(keys[prev:i+1], 0, 'A', '', result)
        prev = i+1
        total += min(shortestSeq(seq, depth-1) for seq in result)
    memo[(keys, depth)] = total
    return total

def solve(sequence, maxDepth):
    total = 0
    for code in sequence:
        kpadSeqs = []
        buildKeypadSeq(code, 0, 'A', '', kpadSeqs)
        minlength = min(shortestSeq(seq, maxDepth) for seq in kpadSeqs)
        total += minlength * int(re.sub(r'\D+', '', code))        
    return total

print(solve(sequence, 25))

154517692795352
